In [3]:
# import needed packages
import pandas as pd
import requests
from bs4 import BeautifulSoup
import math
import re
import time

In [6]:
# get id function
def get_id(term):
    id_list = []
    API_KEY = "59cbd6ff6b3ce5d506f0eeb69e4e09362409"
    url_search = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=gds&api_key={API_KEY}&term={term}&retmax=200"
    info = BeautifulSoup(requests.get(url_search).text, 'lxml')
    
    term_count = int(info.find('count').text)
    id_list += [single_term.text for single_term in info.find_all('id')]
    
    for page in range(1, math.ceil(term_count/200)):
        page_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=gds&api_key={API_KEY}&term={term}&retmax=200&retstart={page*200}"
        page_info = BeautifulSoup(requests.get(page_url).text, 'lxml')
        id_list += [single_term.text for single_term in page_info.find_all('id')]
    
    return id_list, term_count


In [7]:
term = "(PBMC)+AND+(single-cell)"
id_list, term_count = get_id(term=term)

/home/ouyangkang/software/anaconda3/envs/ML/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [10]:
# check result
print(len(set(id_list)) == term_count)

True


In [ ]:
API_KEY = "59cbd6ff6b3ce5d506f0eeb69e4e09362409"
id_search = ",".join(id_list[:20])
id_url = id_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=gds&api_key={API_KEY}&id={id_search}&rettype=summary"

NameError: name 'API_KEY' is not defined

In [15]:
id_search

'200246456,200233315,200229626,200233276,200233275,200232117,200237254,200237579,200237562,200189050,200185712,200239626,200215441,200229406,200241835,200241834,200243629,200239915,200239909,200239903'